In [ ]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn import model_selection

DATADIR = Path('./data')
WORKDIR = Path('./work')

In [ ]:
context_json = json.load(open(DATADIR / 'context.json'))  # list of strings
train_json = json.load(open(DATADIR / 'train.json'))
valid_json = json.load(open(DATADIR / 'valid.json'))

len(context_json), len(train_json), len(valid_json)

(9013, 21714, 3009)

In [ ]:
def convert_example_to_swag(example, contexts):
    conv = {
        'sent1': example['question'],
        'sent2': '',  # will be merely prepended to each ending so can be empty
        'ending0': contexts[example['paragraphs'][0]],
        'ending1': contexts[example['paragraphs'][1]],
        'ending2': contexts[example['paragraphs'][2]],
        'ending3': contexts[example['paragraphs'][3]],
    }
    if 'relevant' in example:
        conv['label'] = example['paragraphs'].index(example['relevant'])
    return conv

# Test on one example
train_json[0], '-'*100,\
convert_example_to_swag(train_json[0], context_json)

({'id': '593f14f960d971e294af884f0194b3a7',
  'question': '舍本和誰的數據能推算出連星的恆星的質量？',
  'paragraphs': [2018, 6952, 8264, 836],
  'relevant': 836,
  'answer': {'text': '斯特魯維', 'start': 108}},
 '----------------------------------------------------------------------------------------------------',
 {'sent1': '舍本和誰的數據能推算出連星的恆星的質量？',
  'sent2': '',
  'ending0': '1930年，印度物理學家蘇布拉馬尼揚·錢德拉塞卡根據廣義相對論計算出質量大於1.4倍太陽質量的非轉動星體會因重力塌縮成為電子簡併態。愛丁頓雖然在理論上支持黑洞存在的可能性，但同時認為錢德拉塞卡的觀點事實上不能成立，他認為「應當有某種自然定律阻止恆星出現這種荒唐的行為」。當時的物理學家如波耳、亨利·羅素等人都贊同錢德拉塞卡的理論，但出於愛丁頓聲望的原因，他們並沒有公開對錢德拉塞卡表示支持。不過從某種意義上說，愛丁頓也是正確的，當恆星質量大於錢德拉塞卡極限後，確實仍然會有中子簡併壓力阻止恆星繼續塌縮。到了1939年，美國物理學家羅伯特·歐本海默等人推算了這種情形的恆星質量上限，這個極限被稱作托爾曼-歐本海默-沃爾科夫極限。當今的天體物理學家普遍認為，除非有如未知的夸克簡併壓力一類因素的存在，質量大於托爾曼-歐本海默-沃爾科夫極限的恆星將最終會塌縮為錢德拉塞卡所預言的黑洞。即使如此，史瓦西解作為當時能夠描述黑洞行為的唯一精確解，由於具有一些讓人看似不大優美的性質以及難以與實驗觀測相聯繫，一直沒有進入主流物理學研究的視野，關於黑洞的理論乃至整個廣義相對論領域的研究由此擱置了二十年之久。',
  'ending1': '心理學是否為自然科學的範圍，目前也尚存爭議，一般較廣為接受的說法是心理學同時包含在自然科學與社會科學的範疇之中。自然科學的根本目的在於尋找隱藏在自然現象背後的規律，但是自然科學的工作尚不包括研究為什麼會存在這些規律。自然科學認為超自然的、隨意的和自相矛盾的現象是

In [ ]:
def convert_dataset_to_swag(examples, contexts, path):
    conv = [convert_example_to_swag(ex, contexts) for ex in examples]
    path = Path(path).absolute()
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w') as fp:
        json.dump(conv, fp, indent=2, ensure_ascii=False)

convert_dataset_to_swag(train_json, context_json, WORKDIR / 'train_swag.json')
convert_dataset_to_swag(valid_json, context_json, WORKDIR / 'valid_swag.json')

In [ ]:
Path(WORKDIR / 'train_swag.json').absolute()

PosixPath('/home/guest/r11922a05/ADL/ADL21-HW2/work/train_swag.json')

## Convert dataset to SQUAD format for QA subtask

Ref: https://huggingface.co/datasets/squad_v2

In [ ]:
def convert_example_to_squad(example, contexts):
    assert contexts[example['relevant']][example['answer']['start']:].startswith(example['answer']['text'])
    return {
        'id': example['id'],
        'title': example['id'],
        'context': contexts[example['relevant']],
        'question': example['question'],
        'answers': {
            'text': [example['answer']['text']],
            'answer_start': [example['answer']['start']],
        },
    }

# Test on one example
train_json[0], '-'*100,\
convert_example_to_squad(train_json[0], context_json)

({'id': '593f14f960d971e294af884f0194b3a7',
  'question': '舍本和誰的數據能推算出連星的恆星的質量？',
  'paragraphs': [2018, 6952, 8264, 836],
  'relevant': 836,
  'answer': {'text': '斯特魯維', 'start': 108}},
 '----------------------------------------------------------------------------------------------------',
 {'id': '593f14f960d971e294af884f0194b3a7',
  'title': '593f14f960d971e294af884f0194b3a7',
  'context': '在19世紀雙星觀測所獲得的成就使重要性也增加了。在1834年，白塞爾觀測到天狼星自行的變化，因而推測有一顆隱藏的伴星；愛德華·皮克林在1899年觀測開陽週期性分裂的光譜線時發現第一顆光譜雙星，週期是104天。天文學家斯特魯維和舍本·衛斯里·伯納姆仔細的觀察和收集了許多聯星的資料，使得可以從被確定的軌道要素推算出恆星的質量。第一個獲得解答的是1827年由菲利克斯·薩瓦里透過望遠鏡的觀測得到的聯星軌道。對恆星的科學研究在20世紀獲得快速的進展，相片成為天文學上很有價值的工具。卡爾·史瓦西發現經由比較視星等和攝影星等的差別，可以得到恆星的顏色和它的溫度。1921年，光電光度計的發展可以在不同的波長間隔上非常精密的測量星等。阿爾伯特·邁克生在虎克望遠鏡第一次使用干涉儀測量出恆星的直徑。',
  'question': '舍本和誰的數據能推算出連星的恆星的質量？',
  'answers': {'text': ['斯特魯維'], 'answer_start': [108]}})

In [ ]:
def convert_dataset_to_squad(examples, contexts, path):
    conv = [convert_example_to_squad(ex, contexts) for ex in examples]
    conv = {'data': conv}
    path = Path(path).absolute()
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w') as fp:
        json.dump(conv, fp, indent=2, ensure_ascii=False)

convert_dataset_to_squad(train_json, context_json, WORKDIR / 'train_squad.json')
convert_dataset_to_squad(valid_json, context_json, WORKDIR / 'valid_squad.json')